# ALBERT Model
### Imports

In [1]:
import os, re, math, random, json, string, pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

import transformers
from transformers import AlbertForTokenClassification, AlbertTokenizerFast
from transformers import DataCollatorForTokenClassification

from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datasets import load_dataset, ClassLabel, Sequence, load_metric

from sklearn.metrics import accuracy_score, f1_score, precision_score

### Constants

In [2]:
TRAIN = 1
TRAIN_SPLIT = 0.90
RANDOM_SEED = 42
BATCH_SIZES = 16
EPOCHS = 25

json_file_path = "../data/cuad-v1-annotated.json"

### Data Loading

In [3]:
datasets = load_dataset('json', data_files=json_file_path, field='data')

# Create train and validation datasets
datasets = datasets['train'].train_test_split(test_size=1-TRAIN_SPLIT, seed=RANDOM_SEED)
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'split_tokens', 'id'],
        num_rows: 282
    })
    test: Dataset({
        features: ['ner_tags', 'split_tokens', 'id'],
        num_rows: 32
    })
})


### Helper Functions

In [4]:
label_list=['B-AGMT_DATE','B-DOC_NAME','B-PARTY','I-AGMT_DATE','T-DOC_NAME','I-PARTY','0']

In [5]:
def show_elements(dataset, num_examples=1):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))


In [6]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

### Tokenizer & Model 

In [7]:
tokenizer = AlbertTokenizerFast.from_pretrained('albert/albert-base-v2') #add_prefix_space=True
model = AlbertForTokenClassification.from_pretrained('albert/albert-base-v2', num_labels=len(label_list))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

C:\Users\john\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\john\.cache\huggingface\hub\models--albert--albert-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Data Encoding

In [8]:
def encode_dataset(dataset):
    encodings = tokenizer(dataset['split_tokens'], is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, return_tensors="pt")
    labels = [label for label in dataset['ner_tags']]
    encodings.pop('offset_mapping')
    return encodings, labels

In [9]:
train_encodings, train_labels = encode_dataset(datasets['train'])
test_encodings, test_labels = encode_dataset(datasets['test'])

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

### Data Batching

In [10]:
data_collator = DataCollatorForTokenClassification(tokenizer)

### Training

In [11]:
training_args = TrainingArguments(
    output_dir='./results/ALBERT',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZES,
    per_device_eval_batch_size=BATCH_SIZES*4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to=[], 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator, 
)

trainer.train()

C:\Users\john\AppData\Local\Temp/ipykernel_9340/136545975.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,1.959400
20,1.588100
30,0.994200
40,0.622400
50,0.552000
60,0.476100
70,0.433400
80,0.421300
90,0.394700
100,0.362600


TrainOutput(global_step=450, training_loss=0.33484904607137045, metrics={'train_runtime': 450.1983, 'train_samples_per_second': 15.66, 'train_steps_per_second': 1.0, 'total_flos': 115005086620200.0, 'train_loss': 0.33484904607137045, 'epoch': 25.0})

In [12]:
# Save Training Loss by Step
fp = open("training_log_ALBERT", 'wb') 
pickle.dump(trainer.state.log_history, fp)

## Evaluation:
### Test-set

In [13]:
test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)
model.eval()
true_labels = []
pred_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        predictions = predictions.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()

        true_labels.extend(labels.flatten())
        pred_labels.extend(predictions.flatten())
        
# Evaluation Metrics
accuracy_test = accuracy_score(true_labels, pred_labels)
f1_score_test = f1_score(true_labels, pred_labels,average=None)
precision_test = precision_score(true_labels, pred_labels,average=None)

# Print Metrics
print("Test Accuracy:", accuracy_test)
print("Test F1:")
print(np.round(f1_score_test,3))
print("Test Precision:")
print(np.round(precision_test,3))

C:\Users\john\AppData\Local\Temp/ipykernel_9340/136545975.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Test Accuracy: 0.5319251543209876
Test F1:
[0.    0.13  0.547 0.036 0.541 0.57  0.244 0.692]
Test Precision:
[0.    0.333 0.743 0.087 0.584 0.478 0.24  0.541]


C:\Users\john\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Train-set

In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=32, collate_fn=data_collator)
model.eval()
true_labels = []
pred_labels = []

with torch.no_grad():
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        predictions = predictions.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()

        true_labels.extend(labels.flatten())
        pred_labels.extend(predictions.flatten())
        
# Evaluation Metrics
accuracy_train = accuracy_score(true_labels, pred_labels)
f1_score_train = f1_score(true_labels, pred_labels, average=None)
precision_train = precision_score(true_labels, pred_labels, average=None)

# Print Metrics
print("Train Accuracy:", accuracy_train)
print("Train F1:")
print(np.round(f1_score_train,3))
print("Train Precision:")
print(np.round(precision_train,3))

C:\Users\john\AppData\Local\Temp/ipykernel_9340/136545975.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Train Accuracy: 0.4902622987729371
Train F1:
[0.    0.368 0.632 0.253 0.846 0.45  0.73  0.653]
Train Precision:
[0.    0.742 0.691 0.554 0.783 0.298 0.624 0.488]


C:\Users\john\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
